### rcs eddies ###

In [1]:
# longitude range setting
# from -180 to 180
def format(longitude):
    if longitude > 180:
        longitude = longitude - 360
    else:
        longitude = longitude
    return longitude

### pymongo ###

In [2]:
from pymongo import MongoClient
from tqdm import tqdm
import numpy as np
import pandas as pd

In [3]:
# collection
rcs_eddies = MongoClient().eddies.rcs_eddies
rcs_cchdo = MongoClient().eddies.rcs_cchdo

In [4]:
for document in tqdm(rcs_eddies.find()):
    obs_num = len(document['features'][2]['geometry']['coordinates'])
    for i in range(obs_num):
        lon = format(float(document['features'][2]['geometry']['coordinates'][i][0]))
        lat = float(document['features'][2]['geometry']['coordinates'][i][1])
        eddy = {
            'eid': str(document['_id']) + '-' + str(i+1),
            'dat': pd.to_datetime(document['date_start']) + np.timedelta64(i, 'D'),
            'loc': {
                "type": "Point",
                "coordinates": [lon, lat]
            },
            'rad': (document['area']/np.pi)**0.5
        }
        rcs_cchdo.insert_one(dict(eddy))

47690it [09:32, 83.34it/s]
